# Implement PCA from the scratch 
### NUS Statistics society 
### Workshop team

[The data information](http://lib.stat.cmu.edu/datasets/boston)

In [26]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston_dataset = load_boston()
X = pd.DataFrame(boston_dataset.data,columns = boston_dataset.feature_names)
Y = boston_dataset.target
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


Variables(features) in order:
 - __CRIM__    :per capita crime rate by town
 - __ZN__      :proportion of residential land zoned for lots over 25,000 sq.ft.
 - __INDUS__   :proportion of non-retail business acres per town
 - __CHAS__    :Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
 - __NOX__     :nitric oxides concentration (parts per 10 million)
 - __RM__      :average number of rooms per dwelling
 - __AGE__     :proportion of owner-occupied units built prior to 1940
 - __DIS__     :weighted distances to five Boston employment centres
 - __RAD__     :index of accessibility to radial highways
 - __TAX__     :full-value property-tax rate per $10000
 - __PTRATIO__  :pupil-teacher ratio by town
 - __B__        :1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
 - __LSTAT__    :% lower status of the population

> __Step 1 : Standardised the data__

PCA is largely infuenced by scales and different variables have different scales.So it's beter to standard the data before finding PCA components.
We just use the [Sklearn's standardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) scales data of zero mean and unit variance.

The basic ideas is :

$z = \frac{x-\mu}{\sigma}$ where $\mu$ is the mean of the training samples and $\sigma$ is the standard deviation of the training samples.


In [16]:
# data process - standardised the data
from sklearn.preprocessing import StandardScaler
standardised_data = StandardScaler().fit_transform(X)
print(standardised_data.shape)

(506, 13)


In [23]:
from pandas import DataFrame
df = DataFrame(standardised_data)
df.columns = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.413229,-0.487722,0.115738,-0.272599,0.158124,0.439316,0.018673,-0.625796,-0.982843,-0.803212,1.176466,0.387217,-0.418147
502,-0.415249,-0.487722,0.115738,-0.272599,0.158124,-0.234548,0.288933,-0.716639,-0.982843,-0.803212,1.176466,0.441052,-0.500850
503,-0.413447,-0.487722,0.115738,-0.272599,0.158124,0.984960,0.797449,-0.773684,-0.982843,-0.803212,1.176466,0.441052,-0.983048
504,-0.407764,-0.487722,0.115738,-0.272599,0.158124,0.725672,0.736996,-0.668437,-0.982843,-0.803212,1.176466,0.403225,-0.865302


> __Step 2: Compute the Covariance matrix: C__ : 

By math defination : Covariance matrix $C = X^{T}*X$, we can use __numpy matmul()__ function in python:



In [28]:
# find the covariance matrix : C = X.T  * X 
covariance_matrix = np.matmul(standardised_data.T, standardised_data)
print("The shape of covariance_matrix is ", covariance_matrix.shape)

The shape of covariance_matrix is  (13, 13)


In [30]:
DataFrame(covariance_matrix )

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,506.000000,-101.437425,205.731206,-28.281141,213.011686,-110.938832,178.483531,-192.113064,316.505604,294.878742,146.712463,-194.842355,230.544469
1,-101.437425,506.000000,-270.117062,-21.604540,-261.401476,157.867237,-288.185895,336.190561,-157.845600,-159.169042,-198.189345,88.813281,-208.975255
2,205.731206,-270.117062,506.000000,31.846642,386.407632,-198.187981,326.257927,-358.261656,301.135413,364.704651,193.923264,-180.630127,305.522657
3,-28.281141,-21.604540,31.846642,506.000000,46.148620,46.173120,43.777994,-50.182945,-3.728330,-18.006778,-61.486678,24.686973,-27.288225
4,213.011686,-261.401476,386.407632,46.148620,506.000000,-152.907223,370.123873,-389.230437,309.388925,338.019739,95.599935,-192.305623,298.984734
5,-110.938832,157.867237,-198.187981,46.173120,-152.907223,506.000000,-121.574055,103.854584,-106.182414,-147.776203,-179.883756,64.802729,-310.586986
6,178.483531,-288.185895,326.257927,43.777994,370.123873,-121.574055,506.000000,-378.427554,230.747361,256.266530,132.326596,-138.408192,304.783296
7,-192.113064,336.190561,-358.261656,-50.182945,-389.230437,103.854584,-378.427554,506.000000,-250.261492,-270.422382,-117.630094,147.504907,-251.479890
8,316.505604,-157.845600,301.135413,-3.728330,309.388925,-106.182414,230.747361,-250.261492,506.000000,460.575463,235.159036,-224.872885,247.270225
9,294.878742,-159.169042,364.704651,-18.006778,338.019739,-147.776203,256.266530,-270.422382,460.575463,506.000000,233.191636,-223.554851,275.260666


> __Step 3: Code to find the eigenvalue and eigenvector__:

We are using scipy.linalg, which have __eigh function__ for us to find the top eigenvalues and eigenvectors:

[More infor on eigh() function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html)

In [35]:
## for example : we are going to find the top 2 eigenvalues and eigenvectors
from scipy.linalg import eigh

# The parameters "eigvals" defined (low to high value)
#eigh function will return the eigenvalue in asending order
#this following code only generat the top 2 eigenvalues
values,vectors = eigh(covariance_matrix,eigvals=(11,12)) # give us the top 2 eigenvalues and eigenvectors
values

array([ 725.23721183, 3100.18550618])

In [36]:
vectors

array([[ 0.31525237,  0.2509514 ],
       [ 0.3233129 , -0.25631454],
       [-0.11249291,  0.34667207],
       [-0.45482914,  0.00504243],
       [-0.21911553,  0.34285231],
       [-0.14933154, -0.18924257],
       [-0.31197778,  0.3136706 ],
       [ 0.34907   , -0.32154387],
       [ 0.27152094,  0.31979277],
       [ 0.23945365,  0.33846915],
       [ 0.30589695,  0.20494226],
       [-0.23855944, -0.20297261],
       [ 0.07432203,  0.30975984]])

In [38]:
print('The shape of the eigen vectors:', vectors.shape )

The shape of the eigen vectors: (13, 2)


In [40]:
# converting the vectors into (2,d) shape for further computation
vectors = vectors.T 
DataFrame(vectors)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.315252,0.323313,-0.112493,-0.454829,-0.219116,-0.149332,-0.311978,0.349070,0.271521,0.239454,0.305897,-0.238559,0.074322
1,0.250951,-0.256315,0.346672,0.005042,0.342852,-0.189243,0.313671,-0.321544,0.319793,0.338469,0.204942,-0.202973,0.309760


- here <code>vectors[1]</code> represents the eigen vector corresponding __1st principle eigen vector__
- here <code>vectors[0]</code> represents the eigen vector corresponding __2nd principle eigen vector__

> __Step 4 : Formating the Principle Compnent__

 Below is the code for Formating the Principle Compnent,by vector-vector multiplication 

In [59]:
# projecting the original data on the plane(the two component)
new_coordinates = np.matmul(vectors, standardised_data.T)
DataFrame(new_coordinates)

,0,1,2,3,4,5,6,7,8,9,...,496,497,498,499,500,501,502,503,504,505
0,-0.773113,-0.591985,-0.599639,0.006871,-0.097712,0.009487,-0.349872,-0.577800,-0.342518,-0.316201,...,-0.259072,-0.378011,-0.460694,-0.434038,-0.593218,-0.724285,-0.759308,-1.155246,-1.041362,-0.761978
1,-2.098297,-1.457252,-2.074598,-2.611504,-2.458185,-2.214852,-1.358881,-0.842045,-0.179928,-1.074184,...,0.669273,0.214016,0.116065,0.425641,0.321704,-0.314968,-0.110513,-0.312360,-0.270519,-0.125803


In [60]:
print("Results new data points' shape :", vectors.shape,"X",standardised_data.shape,"=",new_coordinates.shape)

Results new data points' shape : (2, 13) X (506, 13) = (2, 506)


In [61]:
##appending label to the 2d projected data
new_coordinates = np.vstack((new_coordinates,Y)).T

## for better visualization 
dataframe = pd.DataFrame(data = new_coordinates, columns=('1_st Principles','2_nd Principles','label') )
print("*"*100)
print(dataframe.head(10))

****************************************************************************************************
   1_st Principles  2_nd Principles  label
0        -0.773113        -2.098297   24.0
1        -0.591985        -1.457252   21.6
2        -0.599639        -2.074598   34.7
3         0.006871        -2.611504   33.4
4        -0.097712        -2.458185   36.2
5         0.009487        -2.214852   28.7
6        -0.349872        -1.358881   22.9
7        -0.577800        -0.842045   27.1
8        -0.342518        -0.179928   16.5
9        -0.316201        -1.074184   18.9
